In [1]:
import pandas as pd
import datetime
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

### Load Data

In [2]:
df = pd.read_csv('./Data/Police_Incidents.csv')
print(df.shape)

(110861, 9)


In [3]:
df.isnull().sum()

Incident Number       0
Date Occurred         4
UCR                   4
UCR Description      24
Location            346
Gang Related          4
Latitude           2274
Longitude          2274
Map Point          2274
dtype: int64

### Handle Null Values

Note that the number of null latitude values is has double the percent of crimes post line opening (~30%) vs. the total dataset (~14%). The relatively small amount of null values does allow for the exclusion without substantially changing the crime split pre and post line opening, but is something to keep in mind

In [4]:
# remove null date occured and convert to datetime
df_clean = df[df['Date Occurred'].notnull()]
df_clean['Date Occurred'] = pd.to_datetime(df_clean['Date Occurred'])

# remove null latitude (and by extension) longitude values
df_clean = df_clean[df_clean['Latitude'].notnull()]

# move the UCR to UCR description for those records with null values for UCR description
df_clean.loc[df_clean['UCR Description'].isnull(), 'UCR Description'] = df_clean[df_clean['UCR Description'].isnull()]['UCR'].tolist()

### Create Features

In [5]:
# create flag for line open or not
open_date = datetime.datetime(2016, 5, 20)
df_clean['Line_Open'] = df_clean['Date Occurred'] >= open_date

# add month, year, day of week features
df_clean['Month'] = df_clean['Date Occurred'].dt.month
df_clean['Year'] = df_clean['Date Occurred'].dt.year
df_clean['Day_Of_Week'] = df_clean['Date Occurred'].dt.dayofweek

# create delta between crime and open
df_clean['Day_Delta'] = (df_clean['Date Occurred'] - open_date).dt.days

# create aggregate categories
categories = {'Public Intoxication': ['Public Intoxication', 'Non-Violent'],
 'Larceny - Bicycle': ['Larceny', 'Non-Violent'],
 'Robbery-Strongarm-Commercial': ['Robery', 'Violent'],
 'Municipal Code Violation': ['Municipal Code Violation', 'Non-Violent'],
 'Simple Assault': ['Assault', 'Violent'],
 'Larceny - Shoplift': ['Larceny', 'Non-Violent'],
 'Vandalism': ['Vandalism', 'Non-Violent'],
 'Burglary-Force Resd': ['Burglary', 'Non-Violent'],
 'Viol of Regulatory Laws': ['Viol of Regulatory Laws', 'Non-Violent'],
 'Violation of Quarantine': ['Other', 'Non-Violent'],
 'Simple Assault - Part II Crime': ['Assault', 'Violent'],
 'Larceny - From Vehicle': ['Larceny', 'Non-Violent'],
 'Fraud': ['Fraud', 'Non-Violent'],
 'Larceny - Other': ['Larceny', 'Non-Violent'],
 'Burglary-Force Non-Resd': ['Burglary', 'Non-Violent'],
 'Agg Assault - Othr Wpn': ['Agg Assault', 'Violent'],
 'Larceny - From Building': ['Larceny', 'Non-Violent'],
 'DUI': ['DUI', 'Non-Violent'],
 'Arson-Motor Vehicle': ['Arson', 'Non-Violent'],
 'GTA - Passenger Car': ['GTA', 'Non-Violent'],
 'Contempt of Court': ['Contempt of Court', 'Non-Violent'],
 'GTA - Other Vehicle': ['GTA', 'Non-Violent'],
 'Burglay-Unlaw Entry-Resd': ['Burglary', 'Non-Violent'],
 'Embezzlement': ['Embezzlement', 'Non-Violent'],
 'Larceny - Vehicle Parts/Acc': ['Larceny', 'Non-Violent'],
 'Recv/Poss Stolen Property': ['Recv/Poss Stolen Property', 'Non-Violent'],
 'Vagrancy': ['Vagrancy', 'Non-Violent'],
 'Runaway': ['Runaway', 'Other / Unknown'],
 'GTA - Recovery': ['GTA', 'Non-Violent'],
 'Disorderly Conduct': ['Disorderly Conduct', 'Non-Violent'],
 'Robbery Strongarm-Street/Hwy': ['Robery', 'Violent'],
 'Narco Possess-Synthetic': ['Narcotics', 'Non-Violent'],
 'Narco Possess-Marijuana': ['Narcotics', 'Non-Violent'],
 'Robbery-Strongarm-Other Loc': ['Robery', 'Violent'],
 'Arson-Single Fam Resd-Inhab': ['Arson', 'Non-Violent'],
 'Trespass/Illegal Entry': ['Trespass/Illegal Entry', 'Non-Violent'],
 'Agg Assault - Hands': ['Agg Assault', 'Violent'],
 'Burglay-Unlaw Entry Non-Resd': ['Burglary', 'Non-Violent'],
 'Liquor Law Violations': ['Liquor Law Violations', 'Non-Violent'],
 'Forgery/Counterfeit': ['Forgery/Counterfeit', 'Non-Violent'],
 'All Other Offenses': ['Other', 'Other / Unknown'],
 'Narco Possess-Other': ['Narcotics', 'Non-Violent'],
 'Contributing Delinq Minor': ['Contributing Delinq Minor', 'Non-Violent'],
 'Carry/Poss Weapon': ['Weapons', 'Non-Violent'],
 'Family Offenses': ['Family Offenses', 'Non-Violent'],
 'Agg Assault - Knife': ['Agg Assault', 'Violent'],
 'Burglary Attempt - Non-Resd': ['Burglary', 'Non-Violent'],
 'Juvenile Curfew/Loitering ': ['Juvenile Curfew/Loitering ', 'Non-Violent'],
 'Robbery-Othr Wpn-Other Loc': ['Robery', 'Violent'],
 'Burglary - General': ['Burglary', 'Non-Violent'],
 'Sex Offenses - Other': ['Sex Offenses', 'Violent'],
 'Robbery Strongarm-Residential': ['Robery', 'Violent'],
 'Agg Assault - General': ['Agg Assault', 'Violent'],
 'Robbery-Firearm-Street/Hwy': ['Robery', 'Violent'],
 'Prostitution': ['Sex Offenses', 'Non-Violent'],
 'Possess Burglary Tool': ['Possess', 'Non-Violent'],
 'Robbery-Knife-Street/Hwy': ['Robery', 'Violent'],
 'Crimes During Incarceration': ['Other', 'Non-Violent'],
 'Larceny - Pickpocket': ['Larceny', 'Non-Violent'],
 'Larceny - Purse-snatch': ['Larceny', 'Non-Violent'],
 'Burglary Attempt - Resd': ['Burglary', 'Non-Violent'],
 'Robbery Strongarm-Bank': ['Robery', 'Violent'],
 'Narco Sales-Marijuana': ['Narcotics', 'Non-Violent'],
 'Possess Drug Paraphernalia': ['Possess', 'Non-Violent'],
 'Other Animal Crimes': ['Animals', 'Non-Violent'],
 'Narco Sales-Synthetic': ['Narcotics', 'Non-Violent'],
 'Robbery-Firearm-Other Loc': ['Robery', 'Violent'],
 'Kidnapping/Abduction': ['Kidnapping/Abduction', 'Violent'],
 'Agg Assault - Firearm': ['Agg Assault', 'Violent'],
 'Robbery-Othr Wpn-Street/Hwy': ['Robery', 'Violent'],
 'GTA - Comm Vehicle': ['GTA', 'Non-Violent'],
 'Arson-Other Comm-Inhab': ['Arson', 'Non-Violent'],
 'Narco Sales Opiate/Cocaine': ['Narcotics', 'Non-Violent'],
 'Robbery-Firearm-Commercial': ['Robery', 'Violent'],
 'Federal Violation': ['Federal Violation', 'Non-Violent'],
 'Robbery-Strongarm-Gas Station': ['Robery', 'Violent'],
 'Arson-All Others': ['Arson', 'Non-Violent'],
 'Blackmail/Extortion': ['Blackmail/Extortion', 'Non-Violent'],
 'Possess Obscene Literature': ['Possess', 'Non-Violent'],
 'Robbery-Othr Wpn Conv Store': ['Robery', 'Violent'],
 'Arson-Other Comm-Uninhab': ['Arson', 'Non-Violent'],
 'Larceny - From Coin Machine': ['Larceny', 'Non-Violent'],
 'Homicide': ['Homicide', 'Violent'],
 'Robbery-Knife-Other Loc': ['Robery', 'Violent'],
 'Robbery-Strongarm Store': ['Robery', 'Violent'],
 'Cruelty to Animals': ['Animals', 'Non-Violent'],
 'Unfair Business Practices': ['Other', 'Non-Violent'],
 'Robbery-Knife-Commercial': ['Robery', 'Violent'],
 'Robbery-Firearm-Residential': ['Robery', 'Violent'],
 'Crime Syndicates': ['Other', 'Non-Violent'],
 'Robbery-Knife-Residential': ['Robery', 'Violent'],
 'Narco Possess-Unspecified Drug': ['Narcotics', 'Non-Violent'],
 'Arson-Other Resd-Uninhab': ['Arson', 'Non-Violent'],
 'Misappropriation of Property': ['Misappropriation of Property',
  'Non-Violent'],
 'Arson-Other Structure-Uninhab': ['Arson', 'Non-Violent'],
 'Telecom/Utility Violations': ['Other', 'Non-Violent'],
 'Public Nuisances': ['Public Nuisances', 'Non-Violent'],
 'Narco Possess-Opiate/Cocaine': ['Narcotics', 'Non-Violent'],
 'Robbery-Firearm-Bank': ['Robery', 'Violent'],
 'Possess/Use Explosives': ['Possess', 'Non-Violent'],
 'Arson-Public Bldg-Uninhab': ['Arson', 'Non-Violent'],
 'Robbery-Othr Wpn-Commercial': ['Robery', 'Violent'],
 'Robbery-Knife-Conv Store': ['Robery', 'Violent'],
 'Larceny - General': ['Larceny', 'Non-Violent'],
 'Robbery Othr Wpn-Bank': ['Robery', 'Violent'],
 'Manslaughter': ['Manslaughter', 'Violent'],
 'Arson-Public Bldg-Inhab': ['Arson', 'Non-Violent'],
 'Aiding/Abetting Crimes': ['Other', 'Non-Violent'],
 'Robbery-Firearm Conv Store': ['Robery', 'Violent'],
 'Arson-Single Fam Resd-Uninhab': ['Arson', 'Non-Violent'],
 'Robbery - General': ['Robery', 'Violent'],
 'Robbery-Othr Wpn-Residential': ['Robery', 'Violent'],
 'Arson-Other Structure-Inhab': ['Arson', 'Non-Violent'],
 'Arson-Other Mobile Property': ['Arson', 'Non-Violent'],
 'Telecommunication Violations': ['Other', 'Non-Violent'],
 'Narco Sales-Other': ['Narcotics', 'Non-Violent'],
 'Arson-Other Resd-Inhab': ['Arson', 'Non-Violent'],
 'Narcotics - General': ['Narcotics', 'Non-Violent'],
 'Robbery-Firearm-Gas Station': ['Robery', 'Violent'],
 'Narco-Sales Unspecified Drug': ['Narcotics', 'Non-Violent'],
 'Gambling - General': ['Other', 'Non-Violent'],
 'Robbery-Othr Wpn-Gas Station': ['Robery', 'Violent'],
 'Robbery-Knife-Bank': ['Robery', 'Violent'],
 'Arson - General': ['Arson', 'Non-Violent']}

df_clean['UCR Level 2'] = df_clean['UCR Description'].apply(lambda x: categories[x][0])
df_clean['UCR Level 1'] = df_clean['UCR Description'].apply(lambda x: categories[x][1])

In [6]:
print(df_clean.shape)
df_clean.head()

(108587, 16)


,Incident Number,Date Occurred,UCR,UCR Description,Location,Gang Related,Latitude,Longitude,Map Point,Line_Open,Month,Year,Day_Of_Week,Day_Delta,UCR Level 2,UCR Level 1
0,090033416,2009-04-02,1823,Narco Possess-Synthetic,1600BLK OCEAN FRONT WALK,False,34.010298,-118.495739,"(34.0102977457032, -118.495738552074)",False,4,2009,3,-2605,Narcotics,Non-Violent
1,090034125,2009-04-03,2300,Public Intoxication,STEWART ST / PICO BLVD,False,34.024252,-118.460508,"(34.0242518901242, -118.460507893009)",False,4,2009,4,-2604,Public Intoxication,Non-Violent
2,090034163,2009-04-03,1822,Narco Possess-Marijuana,500BLK LINCOLN BLVD,False,34.029983,-118.503646,"(34.0299830211087, -118.503646242239)",False,4,2009,4,-2604,Narcotics,Non-Violent
3,090034227,2009-04-04,2300,Public Intoxication,26TH ST / SANTA MONICA BLVD,False,34.033306,-118.474302,"(34.0333062753406, -118.474302319364)",False,4,2009,5,-2603,Public Intoxication,Non-Violent
4,090034245,2009-04-04,1400,Vandalism,100BLK KINNEY ST,False,33.998859,-118.480934,"(33.9988589911957, -118.480933903635)",False,4,2009,5,-2603,Vandalism,Non-Violent


### Join to Geographic Filters

In [57]:
# Load in data with geo columns
df_geo = pd.read_csv("data/Police_Incidents_service_areas.csv")
print df_geo.shape, df_clean.shape

# Set index for join, drop other columns
df_geo = df_geo.rename(columns={"incident_n":"Incident Number"})
df_geo = df_geo[['one_mile','half_mile', 'Incident Number']]
df_geo = df_geo.set_index('Incident Number')
df_geo.head()

(110861, 11) (108587, 16)


,one_mile,half_mile
Incident Number,,
120107330,0,0
140004851,0,0
170124506,0,0
060025453,0,0
090063765,1,0


In [49]:
# Join on IDs
df_clean_join = df_clean.join(df_geo, on='Incident Number')
print df_clean_join.shape
df_clean_join.head()

(108587, 18)


,Incident Number,Date Occurred,UCR,UCR Description,Location,Gang Related,Latitude,Longitude,Map Point,Line_Open,Month,Year,Day_Of_Week,Day_Delta,UCR Level 2,UCR Level 1,one_mile,half_mile
0,090033416,2009-04-02,1823,Narco Possess-Synthetic,1600BLK OCEAN FRONT WALK,False,34.010298,-118.495739,"(34.0102977457032, -118.495738552074)",False,4,2009,3,-2605,Narcotics,Non-Violent,1.0,1.0
1,090034125,2009-04-03,2300,Public Intoxication,STEWART ST / PICO BLVD,False,34.024252,-118.460508,"(34.0242518901242, -118.460507893009)",False,4,2009,4,-2604,Public Intoxication,Non-Violent,1.0,0.0
2,090034163,2009-04-03,1822,Narco Possess-Marijuana,500BLK LINCOLN BLVD,False,34.029983,-118.503646,"(34.0299830211087, -118.503646242239)",False,4,2009,4,-2604,Narcotics,Non-Violent,0.0,0.0
3,090034227,2009-04-04,2300,Public Intoxication,26TH ST / SANTA MONICA BLVD,False,34.033306,-118.474302,"(34.0333062753406, -118.474302319364)",False,4,2009,5,-2603,Public Intoxication,Non-Violent,1.0,0.0
4,090034245,2009-04-04,1400,Vandalism,100BLK KINNEY ST,False,33.998859,-118.480934,"(33.9988589911957, -118.480933903635)",False,4,2009,5,-2603,Vandalism,Non-Violent,0.0,0.0


In [50]:
# There's one row with a bad ID
df_clean_join[df_clean_join.one_mile.isnull()]

,Incident Number,Date Occurred,UCR,UCR Description,Location,Gang Related,Latitude,Longitude,Map Point,Line_Open,Month,Year,Day_Of_Week,Day_Delta,UCR Level 2,UCR Level 1,one_mile,half_mile
34751,.06007960,2006-08-17,1100,Fraud,1500BLK 14TH ST,False,34.022077,-118.48335,"(34.0220765046201, -118.483349753527)",False,8,2006,3,-3564,Fraud,Non-Violent,NaN,NaN


In [51]:
# Let's fill with 0 for now
df_clean_join.one_mile = df_clean_join.one_mile.fillna(0)
df_clean_join.one_mile = df_clean_join.one_mile.astype('int')
df_clean_join.half_mile = df_clean_join.half_mile.fillna(0)
df_clean_join.half_mile = df_clean_join.half_mile.astype('int')
df_clean_join.head()

,Incident Number,Date Occurred,UCR,UCR Description,Location,Gang Related,Latitude,Longitude,Map Point,Line_Open,Month,Year,Day_Of_Week,Day_Delta,UCR Level 2,UCR Level 1,one_mile,half_mile
0,090033416,2009-04-02,1823,Narco Possess-Synthetic,1600BLK OCEAN FRONT WALK,False,34.010298,-118.495739,"(34.0102977457032, -118.495738552074)",False,4,2009,3,-2605,Narcotics,Non-Violent,1,1
1,090034125,2009-04-03,2300,Public Intoxication,STEWART ST / PICO BLVD,False,34.024252,-118.460508,"(34.0242518901242, -118.460507893009)",False,4,2009,4,-2604,Public Intoxication,Non-Violent,1,0
2,090034163,2009-04-03,1822,Narco Possess-Marijuana,500BLK LINCOLN BLVD,False,34.029983,-118.503646,"(34.0299830211087, -118.503646242239)",False,4,2009,4,-2604,Narcotics,Non-Violent,0,0
3,090034227,2009-04-04,2300,Public Intoxication,26TH ST / SANTA MONICA BLVD,False,34.033306,-118.474302,"(34.0333062753406, -118.474302319364)",False,4,2009,5,-2603,Public Intoxication,Non-Violent,1,0
4,090034245,2009-04-04,1400,Vandalism,100BLK KINNEY ST,False,33.998859,-118.480934,"(33.9988589911957, -118.480933903635)",False,4,2009,5,-2603,Vandalism,Non-Violent,0,0


### Export to CSV

In [52]:
df_clean_join.to_csv('./Data/Police_Incidents_Final.csv', index = False)